<a href="https://colab.research.google.com/github/raissaazaria/FoDS_FP/blob/main/FoDS_FP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FoDS Final Project
Chellshe Love Simrochelle - 2502043040

Raissa Azaria 2502005805

Dataset derived from: https://www.kaggle.com/datasets/farazrahman/earthquake

In [ ]:
!pip install basemap

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from mpl_toolkits.basemap import Basemap
import re
import missingno as msno
import warnings
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from google.colab import files

In [ ]:
#reading dataset
!gdown --id 1rjTf2IPpX4S8uYpyQJvhSStIM_RT4WRB


In [ ]:
df = pd.read_csv("all_month.csv")

warnings.filterwarnings("ignore")

x = df.iloc[:, [0, 1, 2, 3]].values

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
#show all column in dataset
df.columns

In [ ]:
# prints information about dataset
df.info()
df[0:10]

In [ ]:
#returns a DataFrame object where all the values are replaced with a Boolean value True for NULL values, and otherwise False.
df.isnull()

In [ ]:
#showing missing value in each 
msno.matrix(df, color="0.1")
plt.show()

In [ ]:
#returns the number of missing values in the dataset.
df.isnull().sum()

In [ ]:
#removes the rows that contains NULL values.
df.dropna()

In [ ]:
df.fillna(0,inplace=True)

In [ ]:
print(df.duplicated())

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df.describe()

In [ ]:
corr = df.corr()
sns.heatmap(corr)

In [ ]:
df_outcome = pd.crosstab(index=df["mag"],  
                              columns="count")      

df_outcome

In [ ]:
df_outcome = pd.crosstab(index=df["depth"], 
                              columns="count")

df_outcome

In [ ]:
tempDf = df[['depth','mag']].copy()
ms = MinMaxScaler()
tempDf = ms.fit_transform(tempDf)
tempDf = pd.DataFrame(tempDf, columns=['depth','mag'])
tempDf.head()

In [ ]:
inertia = []
for i in range(1,11):
    kMeans = KMeans(n_clusters = i)
    kMeans.fit(tempDf[['depth','mag']])  
    inertia.append(kMeans.inertia_)  
plt.plot(range(1,11), inertia, 'o-')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.grid(alpha = 0.3) 
plt.show()

In [ ]:
ss = []
for i in range(2,11):
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(tempDf[['depth','mag']])
    cluster_labels = kmeans.labels_
    ss.append(silhouette_score(tempDf[['depth','mag']], cluster_labels)) 
plt.plot(range(2,11), ss, 'o-')
plt.xlabel('Number of clusters')
plt.ylabel('silhouette score')
plt.grid(alpha = 0.3)
plt.show()

In [ ]:
fig = plt.figure(figsize=(9,4), tight_layout=True)

for i, data in enumerate([df['depth'], df['mag']]):
    bins = 9 if i == 2 else 30
    ax = plt.subplot(1,2, i + 1)
    sns.histplot(data, bins=bins, kde=True)

In [ ]:
sns.boxplot(data=df['depth'])
plt.title('depth distribution')
plt.show()

In [ ]:
sns.boxplot(data=df['mag'])
plt.title('magnitude distribution')
plt.show()

In [ ]:
kMeans = KMeans(n_clusters=3)
kMeans.fit(tempDf[['depth','mag']]) 
df['cluster2_3'] = kMeans.labels_
df.head()

In [ ]:
plt.figure(figsize = (8,6))
sns.scatterplot(data = df, y = 'depth', x = 'mag', hue = 'cluster2_3', alpha = 0.3, palette = 'magma')
plt.show()

In [ ]:
tempList1 = ['depth','mag']
tempList2 = ['earthquakeType','dataAmount','percentage'] + tempList1
eT = 1
tempList3 = []
dflen = len(df)
for i in df['cluster2_3'].unique().tolist():
    tempList4 = []
    tempList4.append(i)
    typelen = len(df.loc[df['cluster2_3']==i])
    tempList4.append(typelen)
    typepercent = round((typelen/dflen)*100,2)
    tempList4.append(typepercent)
    for j in tempList1:
        q25 = str(round(df[j].loc[df['cluster2_3']==i].min(),2))
        q75 = str(round(df[j].loc[df['cluster2_3']==i].max(),2))
        if (q25==q75):
            tempList4.append(q25)
        else:
            tempList4.append("("+q25+") - ("+q75+")")
    tempList3.append(tempList4)
tempdf = pd.DataFrame(tempList3,columns = tempList2)
tempdf

In [ ]:
tempdf.sort_values(by='percentage',ascending = False)

In [ ]:
sns.countplot(data = df , x = 'cluster2_3', palette = 'magma')
plt.show()

In [ ]:
def background_map():
    plt.figure(figsize=(15,9))
    m = Basemap(projection='geos', resolution=None,width=6E6, height=4E6, lat_0=0, lon_0=70)
    m.shadedrelief(scale=0.7)
    return m

In [ ]:
m = background_map()

In [ ]:
m = background_map()
m.scatter(df['longitude'], df['latitude'], s=0.5 , c=df['mag'], cmap='Reds', latlon=True, alpha=0.1)
plt.colorbar(label='Magnitude')
plt.title('Earthquake')

In [ ]:
high_earthquake = df.loc[df['mag'] > 5.5]

In [ ]:
mms = MinMaxScaler()

In [ ]:
scaled_depth = mms.fit_transform(high_earthquake['depth'].values.reshape(-1,1)).ravel() * 500

In [ ]:
m = background_map()
m.scatter(high_earthquake['longitude'], high_earthquake['latitude'], s=scaled_depth, c=high_earthquake['mag'], cmap='Reds', latlon=True, alpha=0.3)
plt.colorbar(label='Magnitude')
plt.title('Earthquake')
plt.show()